In [46]:
import requests
from urllib.parse import urlencode
import uuid
import hashlib
import base64
from email.utils import formatdate

class EDAN:
    edan_base = 'http://edan.si.edu/metadata/v2.0/collections/search.htm' 
    
    def __init__(self, app_id, app_key):
        self.app_id = app_id
        self.app_key = app_key
        self.session = requests.Session()
    def query(self, q = '', fqs = '',
              start=0, rows=10):
        request_date = formatdate()
        nonce = str(uuid.uuid4())
        query_params = {'q': q,
                        'fqs': fqs,
                        'rows': rows,
                        'start': start}
        query_param_string = urlencode(query_params)
        string_to_sign = '\n'.join([nonce, query_param_string, request_date, 
                                    self.app_key])

        hashed_string = hashlib.sha1(string_to_sign.encode()).hexdigest()
        encoded_string = base64.b64encode(hashed_string.encode())
        request_headers = {'X-AppId': self.app_id,
                           'X-Nonce': nonce,
                           'X-RequestDate': request_date,
                           'X-AuthContent': encoded_string}

        r = self.session.get(self.edan_base, params = query_params, 
                             headers = request_headers)
        response_json = r.json()
        return response_json
    
    def fetch_all(self, q='', fqs='', step=100):
        results = []
        quick_result = self.query(q='',fqs='', rows=1)
        if 'rowCount' in quick_result:
            result_count = quick_result['rowCount']
        if result_count > 0:
            for start in range(0, result_count, step):
                edan_json = self.query(q=q,fqs=fqs, start=start, rows=step)
                edan_rows = edan_json['rows']
                results += edan_rows
        return results

In [33]:
import json

In [34]:
with open('../mercury_remake/edan_api_key.json') as api_json:
    edan_key = json.load(api_json)
print(edan_key['APP_ID'])

OCIO_DATA_SCIENCE


In [35]:
edan = EDAN(edan_key['APP_ID'], edan_key['APP_KEY'])

In [36]:
test_call = edan.query(q="edanmdm:ofeo-sg_2011-0702A", start=10, rows=1)
test_call

{'rows': [],
 'facetQueries': {},
 'facets': {'culture': [],
  'date': [],
  'object_type': [['Living botanical specimens', 1]],
  'online_media_type': [['Images', 1]],
  'online_visual_material': [['true', 1]],
  'place': [],
  'set_name': [['National Museum of American History', 1],
   ['Smithsonian Gardens Tree Collection', 1]],
  'topic': [['Trees', 1]],
  'type': [['edanmdm', 1]],
  'unit_code': [['OFEO-SG', 1]],
  'p.edanmdm.indexedstructured.common_name': [['Northern Red Oak', 1],
   ['Red Oak', 1]],
  'p.edanmdm.indexedstructured.exhibition_building': [],
  'p.edanmdm.indexedstructured.name': [],
  'p.edanmdm.descriptivenonrepeating.data_source': [['Smithsonian Gardens',
    1]],
  'p.edanmdm.indexedstructured.geo_age-era': [],
  'p.edanmdm.indexedstructured.geo_age-series': [],
  'p.edanmdm.indexedstructured.geo_age-stage': [],
  'p.edanmdm.indexedstructured.geo_age-system': [],
  'p.edanmdm.indexedstructured.language': [],
  'p.edanmdm.indexedstructured.occupation': [],
  'p.

In [37]:
fq_test = '["topic:Trees"]'
test_call = edan.query(fqs = fq_test, start=10, rows=5)
test_call

{'rows': [{'id': 'edanmdm-siris_ari_378834',
   'title': 'Painter and Faun, (painting)',
   'unitCode': 'ARI',
   'linkedId': '0',
   'type': 'edanmdm',
   'url': 'edanmdm:siris_ari_378834',
   'content': {'descriptiveNonRepeating': {'record_ID': 'siris_ari_378834',
     'unit_code': 'ARI',
     'title_sort': 'PAINTER AND FAUN PAINTING',
     'record_link': 'https://siris-artinventories.si.edu/ipac20/ipac.jsp?&profile=ariall&source=~!siartinventories&uri=full=3100001~!378834~!0#focus',
     'title': {'label': 'Title', 'content': 'Painter and Faun, (painting)'},
     'metadata_usage': {'access': 'Usage conditions apply', 'text': ''},
     'data_source': 'Art Inventories Catalog, Smithsonian American Art Museum'},
    'indexedStructured': {'object_type': ['Paintings'],
     'name': ['Stettheimer, Florine'],
     'topic': ['Women',
      'Trees',
      'Palette',
      'Art',
      'Self-portraits',
      'Occupations',
      'Portraits',
      "Artist's Brush",
      'Landscapes',
      

In [38]:
fq_test = '["topic:Trees","unit_code:OFEO-SG","online_media_type:Images"]'
test_call = edan.query(fqs = fq_test, start=10, rows=5)
test_call

{'rows': [{'id': 'edanmdm-ofeo-sg_2011-0787A',
   'title': "Prunus serrulata 'Kanzan'",
   'unitCode': 'OFEO-SG',
   'linkedId': '0',
   'type': 'edanmdm',
   'url': 'edanmdm:ofeo-sg_2011-0787A',
   'content': {'descriptiveNonRepeating': {'record_ID': 'ofeo-sg_2011-0787A',
     'online_media': {'mediaCount': 2,
      'media': [{'thumbnail': 'https://ids.si.edu/ids/deliveryService?id=SG-2011-0787A-WIN1-HL',
        'idsId': 'SG-2011-0787A-WIN1-HL',
        'usage': {'access': 'Usage conditions apply', 'text': ''},
        'caption': 'Photographed by: Hannele Lahti',
        'type': 'Images',
        'content': 'https://ids.si.edu/ids/deliveryService?id=SG-2011-0787A-WIN1-HL'},
       {'thumbnail': 'https://ids.si.edu/ids/deliveryService?id=SG-2011-0787A-FAL1-HL',
        'idsId': 'SG-2011-0787A-FAL1-HL',
        'usage': {'access': 'Usage conditions apply', 'text': ''},
        'caption': 'Photographed by: Hannele Lahti',
        'type': 'Images',
        'content': 'https://ids.si.edu/

In [47]:
tree_fq_query = '["topic:Trees","unit_code:OFEO-SG","data_source:Smithsonian Gardens","online_media_type:Images"]'
test_call = edan.query(fqs = tree_fq_query, start=10, rows=5)
test_call

{'rows': [{'id': 'edanmdm-ofeo-sg_2011-0787A',
   'title': "Prunus serrulata 'Kanzan'",
   'unitCode': 'OFEO-SG',
   'linkedId': '0',
   'type': 'edanmdm',
   'url': 'edanmdm:ofeo-sg_2011-0787A',
   'content': {'descriptiveNonRepeating': {'record_ID': 'ofeo-sg_2011-0787A',
     'online_media': {'mediaCount': 2,
      'media': [{'thumbnail': 'https://ids.si.edu/ids/deliveryService?id=SG-2011-0787A-WIN1-HL',
        'idsId': 'SG-2011-0787A-WIN1-HL',
        'usage': {'access': 'Usage conditions apply', 'text': ''},
        'caption': 'Photographed by: Hannele Lahti',
        'type': 'Images',
        'content': 'https://ids.si.edu/ids/deliveryService?id=SG-2011-0787A-WIN1-HL'},
       {'thumbnail': 'https://ids.si.edu/ids/deliveryService?id=SG-2011-0787A-FAL1-HL',
        'idsId': 'SG-2011-0787A-FAL1-HL',
        'usage': {'access': 'Usage conditions apply', 'text': ''},
        'caption': 'Photographed by: Hannele Lahti',
        'type': 'Images',
        'content': 'https://ids.si.edu/

In [48]:
if 'rowCount' in test_call:
    result_count = test_call['rowCount']
result_count

1702

In [51]:
results = []
result_count = 1702
step = 100
for start in range(0, result_count, step):
    print(start)
    edan_json = edan.query(fqs=tree_fq_query, start=start, rows=step)
    edan_rows = edan_json['rows']
    results += edan_rows

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700


In [52]:
results[1672]

{'id': 'edanmdm-ofeo-sg_2011-0075A',
 'title': "Thuja occidentalis 'Degroot's Spire'",
 'unitCode': 'OFEO-SG',
 'linkedId': '0',
 'type': 'edanmdm',
 'url': 'edanmdm:ofeo-sg_2011-0075A',
 'content': {'descriptiveNonRepeating': {'record_ID': 'ofeo-sg_2011-0075A',
   'online_media': {'mediaCount': 3,
    'media': [{'thumbnail': 'https://ids.si.edu/ids/deliveryService?id=SG-2011-0075A-WIN-HL',
      'idsId': 'SG-2011-0075A-WIN-HL',
      'usage': {'access': 'Usage conditions apply', 'text': ''},
      'caption': 'Photographed by: Hannele Lahti',
      'type': 'Images',
      'content': 'https://ids.si.edu/ids/deliveryService?id=SG-2011-0075A-WIN-HL'},
     {'thumbnail': 'https://ids.si.edu/ids/deliveryService?id=SG-2011-0075A-SUM1-HL',
      'idsId': 'SG-2011-0075A-SUM1-HL',
      'usage': {'access': 'Usage conditions apply', 'text': ''},
      'caption': 'Photographed by: Hannele Lahti',
      'type': 'Images',
      'content': 'https://ids.si.edu/ids/deliveryService?id=SG-2011-0075A-SUM

In [54]:
with open('garden_edan_trees.json','w') as json_out:
    json.dump(results, json_out, indent=2)